# The goal of this notebook is to copy over some photos for our hackathon

We will load a pre-made file with the severity labels based on line item estimates. Afterwards, we'll move the files into a given directory to download and then modify locally
and re-upload

In [2]:
pip install clm-azure-utils==0.5.4 --extra-index-url https://kepornefqmvs2ebxgfaetydb3xavf3jpyljmw5dzm4utwuzmfyea@geico.pkgs.visualstudio.com/_packaging/DATASCIENCECLAIMS/pypi/simple

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com, https://****@geico.pkgs.visualstudio.com/_packaging/DATASCIENCECLAIMS/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:

import yaml
import matplotlib.pyplot as plt
import os, sys
from pathlib import Path
import glob
import numpy as np
import pandas as pd
notebook_dir = os.getcwd() # Change this to this nextbook path if you are having issues
parent_dir = '/home/azureuser/cloudfiles/code/Users/DSiu/TLC_Projects/total-loss-image-recognition/'
sys.path.insert(0, parent_dir)
from clmazureutils.workspace import CLMWorkspace, refresh_aml_token
refresh_aml_token()
from clmazureutils.datapath import CLMDatapath
from clmazureutils.util.databricks_util import map_to_sparkpath


Trying interactive login
Writing token to /anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/clmazureutils/ and /home/azureuser/localfiles/


In [64]:
# Load data from ADLS
import time
ws = CLMWorkspace().get_workspace()
container = 'tlirdstprocessed'
orig_path_ = 'image_data/'
output_path = '/home/azureuser/localfiles/code/projects/clir/'

start_time = time.time()
CLMds1 = CLMDatapath(ws, orig_path_, container=container)
path_ = CLMds1.get_path(method='mount', output_folder=output_path)
image_path = path_ 

print(f'Mounted path:  {path_} in {time.time() - start_time} seconds!')


ws = CLMWorkspace().get_workspace()
container = 'tlirdstprocessed'
orig_path_ = 'metadata/'
output_path = '/home/azureuser/localfiles/code/projects/clir/'

start_time = time.time()
CLMds2 = CLMDatapath(ws, orig_path_, container=container)
path_ = CLMds2.get_path(method='mount', output_folder=output_path)
metadata_path2 = path_ 

print(f'Mounted path:  {path_} in {time.time() - start_time} seconds!')



Retrieved Workspace.from_config()
Mounted path:  /tmp/tmp_b8jg8hm/home/azureuser/localfiles/code/projects/clir/tlirdstprocessed/image_data in 1.2772724628448486 seconds!
Retrieved Workspace.from_config()
Stopping mount at ... /tmp/tmpg9r5gkyh/home/azureuser/localfiles/code/projects/clir/tlirdstprocessed/metadata
Mounted path:  /tmp/tmpi_mw5iyj/home/azureuser/localfiles/code/projects/clir/tlirdstprocessed/metadata in 1.2405760288238525 seconds!


In [8]:
sev_file = metadata_path2 + '/severity_classifier_line_estimate_first_V3_filtered_holdout.parquet'
severity_df = pd.read_parquet(sev_file)

In [10]:
corner_filter = severity_df.EPE_LABEL == 'front_driver_corner'
damage_filter = severity_df.ESTIMATE_DOLLAR == 0
mild_filter = severity_df.SEVERITY_LABEL == 'mild'
severe_filter = severity_df.SEVERITY_LABEL == 'severe'



In [16]:
df1 = severity_df[corner_filter & damage_filter].head(100)
df2 = severity_df[corner_filter & mild_filter].head(50)
df3 = severity_df[corner_filter & severe_filter].head(50)

final_df = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)

In [76]:
image_cols = ['PATH','NAME','INCIDENTID','EPE_LABEL','ESTIMATE_DOLLAR','ESTIMATE_NORM','SEVERITY_LABEL','CLAIMNUMBER',
              'dmg_score','corner_score','corner_label', 'VEHICLEMAKE','VEHICLEMODEL','VEHICLEYEAR']

image_df = final_df.loc[:,image_cols]

# Begin copying the data to the projects directory


In [99]:
output_dir = '/home/azureuser/cloudfiles/code/projects/clir/hackathon51/'

def make_severity_folder(name, label):
    output_folder = os.path.join(output_dir, 'image_data', label) 
    os.makedirs(output_folder, exist_ok=True)
    final_folder = os.path.join(output_folder, name.replace('.jpg/', '.jpg'))
    return final_folder
image_df['PATH_TMP'] = image_df['PATH'].apply(lambda x: re.sub("(.*?)TLIRProcessed", image_path, x.replace('image_data/', '')))
image_df['AML_PATH'] = image_df.apply(lambda x: make_severity_folder(x.NAME,x.SEVERITY_LABEL), axis=1)


In [101]:
image_df['AML_PATH'][0]

'/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/0347599110101092___905256526413.jpg'

In [102]:
import shutil
for i,o in zip(image_df['PATH_TMP'], image_df['AML_PATH']):
    print(o)
    shutil.copyfile(i, o)
               

/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/0347599110101092___905256526413.jpg
/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/0467401300101076_173046368_frontdrivercorner_Claims1-B034147F-0000-C018-95D1-B3AC1E6952F9.jpg
/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/8758793420000001_179390682_frontdrivercorner_Claims2-60B0A282-0000-C714-8EBB-A5E4032F1680.jpg
/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/0625427410101027___905301243635.jpg
/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/0592972550101039___905263014292.jpg
/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/8679493650000001___905206984131.jpg
/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/0459286610107016_172160971_frontdrivercorner_Claims1-A02E887E-0000-C317-81A0-9AB6A3C935B7.jpg
/home/azureuser/cloudfiles/c

# Save output parquet file

In [103]:
image_df.to_parquet(output_dir + '/hackathon51_images.parquet')

In [104]:
print(image_df['AML_PATH'][0])
print(os.path.exists(image_df['AML_PATH'][0]))

/home/azureuser/cloudfiles/code/projects/clir/hackathon51/image_data/noDamage/0347599110101092___905256526413.jpg
True
